In [9]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, Concatenate, add, BatchNormalization, Activation
import os
import glob
import rasterio

import tensorflow as tf
from tensorflow.keras.models import Model

In [10]:
def conv_block(x, filters, batchnorm=True):
    conv1 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        conv1 = BatchNormalization(axis=3)(conv1)
    conv1 = Activation('relu')(conv1)    
    conv2 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(conv1)
    if batchnorm is True:
        conv2 = BatchNormalization(axis=3)(conv2)
    conv2 = Activation("relu")(conv2)

    return conv2

In [11]:
def residual_conv_block(x, filters, batchnorm=True):
    conv1 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        conv1 = BatchNormalization(axis=3)(conv1)
    conv1 = Activation('relu')(conv1)    
    conv2 = Conv2D(filters, (3, 3), kernel_initializer='he_normal', padding='same')(conv1)
    if batchnorm is True:
        conv2 = BatchNormalization(axis=3)(conv2)
    conv2 = Activation("relu")(conv2)
        
    #skip connection    
    shortcut = Conv2D(filters, kernel_size=(1, 1), kernel_initializer='he_normal', padding='same')(x)
    if batchnorm is True:
        shortcut = BatchNormalization(axis=3)(shortcut)
    shortcut = Activation("relu")(shortcut)
    respath = add([shortcut, conv2])       
    return respath

In [12]:
def dense_block(inputs, num_filters):
    conv1 = conv_block(inputs, num_filters)
    concat = Concatenate()([inputs, conv1])
    return concat

In [13]:
def residual_unet(input_shape, num_classes):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = residual_conv_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = residual_conv_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = residual_conv_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = residual_conv_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(drop5)
    up6 = Concatenate()([up6, conv4])
    conv6 = residual_conv_block(up6, 512)
    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = Concatenate()([up7, conv3])
    conv7 = residual_conv_block(up7, 256)
    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = Concatenate()([up8, conv2])
    conv8 = residual_conv_block(up8, 128)
    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = Concatenate()([up9, conv1])
    conv9 = residual_conv_block(up9, 64)
    
    # Output
    outputs = Conv2D(num_classes, 1, activation='softmax')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [14]:
def dense_unet(input_shape, num_classes):
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = dense_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = dense_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = dense_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = dense_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    # Bottleneck
    conv5 = Conv2D(1024, 3, activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv5 = Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)
    
    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(drop5)
    up6 = Concatenate()([up6, conv4])
    conv6 = residual_conv_block(up6, 512)
    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = Concatenate()([up7, conv3])
    conv7 = residual_conv_block(up7, 256)
    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = Concatenate()([up8, conv2])
    conv8 = residual_conv_block(up8, 128)
    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = Concatenate()([up9, conv1])
    conv9 = residual_conv_block(up9, 64)
    
    # Output
    outputs = Conv2D(num_classes, 1, activation='softmax')(conv9)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [15]:
def load_data(input_dir, mask_dir):
    input_files = sorted(glob.glob(os.path.join(input_dir, '*.tif')))
    mask_files = sorted(glob.glob(os.path.join(mask_dir, '*.tif')))
    images = []
    masks = []

    for input_file, mask_file in zip(input_files, mask_files):
        # Read input file
        with rasterio.open(input_file) as src:
            img = src.read(2)  # Read the first band assuming it's a single-band image
            # print(img)
            images.append(img)

        # Read mask file
        with rasterio.open(mask_file) as src:
            msk = src.read(1)  # Read the first band assuming it's a single-band image
            masks.append(msk)

    return np.array(images), np.array(masks)

In [16]:
input_dir = './data/'
mask_dir = './output_tif/'

# Load data
images, masks = load_data(input_dir, mask_dir)

# Define input shape and number of classes
print("Images")
print(images)
images = np.expand_dims(images, axis=-1)
input_shape = images.shape[1:]
print(input_shape)
num_classes = 1  # Assuming binary segmentation, adjust accordingly if needed

# Create and compile the model
model = residual_unet(input_shape, num_classes)
for layer in model.layers:
    print(layer.output_shape)

Images
[[[0.03850178 0.0394277  0.04185019 ... 0.05080734 0.04912348 0.05062708]
  [0.04004565 0.03974563 0.04127562 ... 0.04777289 0.04718909 0.05001535]
  [0.04139828 0.04110415 0.0415015  ... 0.04294866 0.0436424  0.0485488 ]
  ...
  [0.04794572 0.03757069 0.03864455 ... 0.03271767 0.03207743 0.03048064]
  [0.03332274 0.02812962 0.03361928 ... 0.04048121 0.03828616 0.03472147]
  [0.0267618  0.02612066 0.03139854 ... 0.04327314 0.04018223 0.03590726]]

 [[0.08710568 0.09416492 0.09497649 ... 0.00826482 0.00855915 0.00990077]
  [0.0897772  0.09723881 0.09654529 ... 0.00897474 0.00948931 0.01106049]
  [0.08326224 0.09220129 0.09141164 ... 0.01007854 0.01025462 0.01163389]
  ...
  [0.03359238 0.0360279  0.03748187 ... 0.04722763 0.04130829 0.03689655]
  [0.03186584 0.03466028 0.03640364 ... 0.04571833 0.04240851 0.03832721]
  [0.03209388 0.03441574 0.03619102 ... 0.03970888 0.03619634 0.0340551 ]]

 [[0.0320792  0.03503213 0.0461979  ... 0.0459112  0.04632825 0.04909702]
  [0.04015223 0

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 936, 1250, 512), (None, 937, 1250, 512)]

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(images, masks, epochs=10, batch_size=4)  # Adjust epochs and batch size as needed

In [ ]:
test_loss, test_accuracy = model.evaluate(images, masks)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)